In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
# http://localhost:8888/notebooks/Documents/AI/scripts/picture_comparison_crop80.ipynbfrom tqdm import tqdm_notebook, tqdm

import numpy as np
from torchvision.models import resnet18
import os
import shutil
import torch
from torch.utils.tensorboard import SummaryWriter
import time
import torch.optim as optim
from torch.utils.data import DataLoader,  TensorDataset
from torchvision import models, datasets, transforms

import torch.nn as nn


In [2]:
class myDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        if self.transform:
            image = self.transform(image)
        # sample = {"image": image, "label": label}

        return image, label

In [3]:
#load dataset in .nps format

data = np.load('crop80_1A0_10A0_5000_5000.npz')
x_images = torch.from_numpy(data['arr_0']/255)
x_images = torch.transpose(x_images, 1, 3)
y_labels = torch.from_numpy(data['arr_1'])
len_labels = data['arr_2']

In [4]:
x_images.shape



torch.Size([10000, 3, 80, 80])

In [13]:
#Here you can change batch size

len_dataset = len(y_labels)   
    
dataset = myDataset(data=x_images.float(), targets=y_labels)
trainset, testset = torch.utils.data.random_split(dataset, [int(len_dataset*0.8), len_dataset-int(len_dataset*0.8)])

train_loader = DataLoader(trainset, batch_size = 50, shuffle = True)
test_loader = DataLoader(testset, batch_size = 50, shuffle = True)

In [14]:
def validate(model,testloader,device):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            labels=labels.squeeze(1)
            # print(predicted)
            # print(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # print(correct / total)

    return correct / total  

In [15]:
def train(model, num_epochs=50, comment = ""):
    writer = SummaryWriter(comment)
    device = torch.device("cpu")
    sum_acc = np.zeros((1,  num_epochs))
    sum_loss = sum_acc.copy()
    model.train()
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) #change learning rate
    best_accuracy = 0
    best_loss = 100
    fig = plt.figure() 

    for epoch in range(num_epochs):
        epoch_start = time.time()

        for img_batch, labels_batch in train_loader:
            optimizer.zero_grad()
    #         print(img_batch.shape)
    #         print(labels_batch.shape)
            output = model(img_batch.to(device))
            loss = criterion(output, labels_batch.to(device).squeeze().long())
            loss.backward()
            optimizer.step()
            images = img_batch.cpu()
            label_nums = output.cpu() 
            accuracy = validate(model,test_loader,device)
#             print(accuracy)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            print('Best accuracy improved')
            torch.save(model.state_dict(), 'model_weights_crop80_1A0_10A0_5000_5000.pth') #name of saved weights
        if best_loss > loss.cpu().item():
            best_loss = loss.cpu().item()
#             print('Best loss improved')

        sum_acc[0, epoch] = accuracy
        sum_loss[0,epoch] = loss  
        writer.add_scalar('Accuracy',accuracy,epoch)
        writer.add_scalar('Loss/train',loss.cpu().item(),epoch)
        epoch_end = time.time()
        print("Epoch: {} Loss: {:.3f} Accuracy: {:.3f} Time: {:.4f}s".format(epoch, loss.item(),accuracy, epoch_end-epoch_start))
        
        plt.plot(sum_acc[0,0:epoch].T)
        plt.show()
        writer.flush()
    writer.close()
    
    return sum_acc, sum_loss

In [16]:
model = resnet18(pretrained=False)
model.conv1.weight = torch.nn.Parameter(torch.rand((64,3,7,7)))
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.float()
print('0')

0


In [ ]:
accuracy, loss = train(model, 100) #change number of epochs

In [ ]:
np.max(accuracy)

In [ ]:
plt.plot(accuracy.T, 'r')

In [ ]:
plt.plot(loss.T, 'b')